In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu11 cuml-cu11
!pip install pandas==1.5.3

In [ ]:
!pip install \
    cuml-cu12 --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12 --extra-index-url=https://pypi.nvidia.com \
    rmm-cu12 --extra-index-url=https://pypi.nvidia.com \
    rapids-dask-dependency --extra-index-url=https://pypi.nvidia.com

print("Cài đặt lại RAPIDS cho CUDA 12 hoàn tất!")

In [ ]:

#split
import os
from abc import ABCMeta, abstractmethod
from numbers import Integral, Real
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from scipy.special import logsumexp
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
import warnings
import pickle

warnings.filterwarnings("ignore", category=RuntimeWarning)

try:
    import cuml
    from cuml.svm import SVC as CuSVC
    print("CuML imported successfully - GPU support enabled!")
    GPU_AVAILABLE = True
except ImportError:
    print("CuML not available, falling back to CPU sklearn")
    from sklearn.svm import SVC as CuSVC
    GPU_AVAILABLE = False

print("Checking GPU availability...")
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader,nounits

def main(folder_path):
    with open("log_result_SVM_split_validation.txt", "a") as log_file:
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".csv"):
                file_path = os.path.join(folder_path, file_name)
                print(f"Processing file: {file_name}")

                cols = ["Destination Port", "Flow Duration", "Total Fwd Packets", "Total Backward Packets",
                        "Total Length of Fwd Packets", "Total Length of Bwd Packets", "Fwd Packet Length Max",
                        "Fwd Packet Length Min", "Fwd Packet Length Mean", "Fwd Packet Length Std",
                        "Bwd Packet Length Max", "Bwd Packet Length Min", "Bwd Packet Length Mean",
                        "Bwd Packet Length Std", "Flow Bytes/s", "Flow Packets/s", "Flow IAT Mean",
                        "Flow IAT Std", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Total", "Fwd IAT Mean",
                        "Fwd IAT Std", "Fwd IAT Max", "Fwd IAT Min", "Bwd IAT Total", "Bwd IAT Mean",
                        "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min", "Fwd PSH Flags", "Bwd PSH Flags",
                        "Fwd URG Flags", "Bwd URG Flags", "Fwd Header Length", "Bwd Header Length",
                        "Fwd Packets/s", "Bwd Packets/s", "Min Packet Length", "Max Packet Length",
                        "Packet Length Mean", "Packet Length Std", "Packet Length Variance", "FIN Flag Count",
                        "SYN Flag Count", "RST Flag Count", "PSH Flag Count", "ACK Flag Count", "URG Flag Count",
                        "CWE Flag Count", "ECE Flag Count", "Down/Up Ratio", "Average Packet Size",
                        "Avg Fwd Segment Size", "Avg Bwd Segment Size", "Fwd Avg Bytes/Bulk",
                        "Fwd Avg Packets/Bulk", "Fwd Avg Bulk Rate", "Bwd Avg Bytes/Bulk", "Bwd Avg Packets/Bulk",
                        "Bwd Avg Bulk Rate", "Subflow Fwd Packets", "Subflow Fwd Bytes", "Subflow Bwd Packets",
                        "Subflow Bwd Bytes", "Init_Win_bytes_forward", "Init_Win_bytes_backward",
                        "act_data_pkt_fwd", "min_seg_size_forward", "Active Mean", "Active Std",
                        "Active Max", "Active Min", "Idle Mean", "Idle Std", "Idle Max", "Idle Min", "Label"]

                df = pd.read_csv(file_path, names=cols)
                df["Label"] = df["Label"].apply(lambda x: 0 if x == "BENIGN" else 1)
                print(df["Label"].value_counts())
                print(df.head())

                # Split data
                train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["Label"], random_state=42)
                train_df, valid_df = train_test_split(train_df, test_size=0.25, stratify=train_df["Label"], random_state=42)

                # Preprocess data
                for subset in [train_df, valid_df, test_df]:
                    for col in subset.columns[:-1]:
                        subset[col] = pd.to_numeric(subset[col], errors='coerce')
                    subset.dropna(inplace=True)

                x_train = train_df[train_df.columns[:-1]].values
                y_train = train_df[train_df.columns[-1]].values
                x_valid = valid_df[valid_df.columns[:-1]].values
                y_valid = valid_df[valid_df.columns[-1]].values
                x_test = test_df[test_df.columns[:-1]].values
                y_test = test_df[test_df.columns[-1]].values

                x_train = np.nan_to_num(x_train, nan=0.0, posinf=0.0, neginf=0.0)
                x_valid = np.nan_to_num(x_valid, nan=0.0, posinf=0.0, neginf=0.0)
                x_test = np.nan_to_num(x_test, nan=0.0, posinf=0.0, neginf=0.0)

                # Standardize features
                scaler = StandardScaler()
                x_train = scaler.fit_transform(x_train)
                x_valid = scaler.transform(x_valid)
                x_test = scaler.transform(x_test)

                print("Training model...")
                svm_model = CuSVC(
                      C=1.0,
                      kernel='rbf',
                      gamma='scale',
                      tol=1e-3,
                      cache_size=500.0,
                      max_iter=-1
                    )


                y_pred = svm_model.predict(x_test)

                log_file.write(f"Result of file: {file_name}\n")
                log_file.write(f"Library: {'CuML (GPU)' if GPU_AVAILABLE else 'sklearn (CPU)'}\n")
                log_file.write(classification_report(y_test, y_pred))
                log_file.write(f"Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%\n")
                log_file.write("Model training complete.\n")
                log_file.write("---------------------------------------------------\n\n")

                print(classification_report(y_test, y_pred))
                print(f"Result of file: {file_name}")
                print(f"Library: {'CuML (GPU)' if GPU_AVAILABLE else 'sklearn (CPU)'}")
                print(f"Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")
                print("Model training complete.")
                print("---------------------------------------------------\n")

if __name__ == "__main__":
    folder_path = "/content/final_dts"
    main(folder_path)